In [1]:
from math import log
from math import exp
import math
import numpy as np
import random
from numpy.linalg import inv
from scipy.stats import t
from scipy.stats import lognorm
import itertools
import time
from scipy.stats import chi2
from scipy.stats import gamma
from random import choices

In [2]:
def psi(X):
    return np.quantile(X,q)

In [5]:
q = 0.7 # target quantile in psi
n_sim = 100000
N = 3000
# number of computations for psi (for batching with equal or unequal batch sizes, K is #batches, 
# for other methods, K is #batches + 1)
K_list = [6, 12, 17] 
gm_list = [0.3]
values = [[2.252],[2.063],[2.041]] 
result = []
for K_idx in range(len(K_list)):
    result_K = []
    for gm_idx in range(len(gm_list)):
        K = K_list[K_idx]
        gm = gm_list[gm_idx]
        c_value = values[K_idx][gm_idx]
        gamma = np.array(list(range(K))) + 1
        gamma = gamma/np.sum(gamma)
        t_quantile = t.ppf(0.95,K-1)

        ct1 = 0
        ct2 = 0
        ct3 = 0
        ct4 = 0
        ct5 = 0
        hl1_list = []
        hl2_list = []
        hl3_list = []
        hl4_list = []
        hl5_list = []

        trueth = lognorm.ppf(q, 1)

        gm = 0.3
        gm_vec = gm*np.ones((K,))
        beta_mtx = np.zeros((K,K))
        for i in range(1,K):
            for j in range(1,K):
                beta_mtx[i,j] = max(0,(gm - np.abs(i-j)*(1-gm)/(K-2)))
        V = np.zeros((K,K))
        for i in range(1,K):
            for j in range(1,K):
                if i==j:
                    V[i,j] = 1 / gm
                else:
                    V[i,j] = beta_mtx[i,j]/(gm**2)
        V[0,0] = 1
        for i in range(1,K):
            V[i,0] = 1
            V[0,i] = 1

        V_inv  = np.linalg.inv(V)
        lm = np.sum(V_inv)
        start = time.time()
        from tqdm import tqdm
        for i in tqdm(range(n_sim)):
            X = np.random.lognormal(size = (N,))

            # batching
            Y = np.zeros((K,))
            for k in range(K):
                Y[k] = psi(X[(k*(N//K)):((k+1)*(N//K))])
            pt1 = np.average(Y)
            hl1 = t_quantile * np.std(Y,ddof=1) / np.sqrt(K)
            if np.abs(trueth-pt1) < hl1:
                ct1 += 1
            hl1_list.append(hl1)

            # batching with unequal length
            Y = np.zeros((K,))
            data_used = 0
            for k in range(K):
                Y[k] = psi(X[data_used:(data_used + int(N*gamma[k]))])
                data_used += int(N*gamma[k])

            pt2 = np.inner(gamma, Y)
            S = np.sqrt(np.inner(gamma, np.power(Y - pt2, 2)))
            hl2 = t_quantile * S / np.sqrt(K - 1)
            if np.abs(trueth-pt2) < hl2:
                ct2 += 1
            hl2_list.append(hl2)     


            # overlapping batching
            Y = np.zeros((K,))
            Y[0] = psi(X)
            for k in range(1,K):
                Y[k] = psi(X[int((k-1)*(N*(1-gm)/(K-2))) : int(((k-1)*(N*(1-gm)/(K-2)) + gm*N))])
            pt3 = np.sum(V_inv.dot(Y.reshape(-1,1)))/lm
            S = np.sqrt(1/lm*(Y-pt3).reshape(1,-1).dot(V_inv.dot((Y-pt3).reshape(-1,1))))
            hl3 = t_quantile * S / np.sqrt(K - 1)
            if np.abs(trueth-pt3) < hl3:
                ct3 += 1
            hl3_list.append(hl3)

            # cheap bootstrap
            Y = np.zeros((K,))
            Y[0] = psi(X)
            for k in range(1,K):
                Xstar = random.choices(X, k=len(X))
                Y[k] = psi(Xstar)
            pt4 = Y[0]
            S = np.sqrt(np.average(np.power(Y[1:]-pt4,2)))
            hl4 = t_quantile * S
            if np.abs(trueth-pt4) < hl4:
                ct4 += 1
            hl4_list.append(hl4)

            # glynn's paper
            Y = np.zeros((K,))
            Y[0] = psi(X)
            for k in range(1,K):
                Y[k] = psi(X[int((k-1)*(N*(1-gm)/(K-2))) : int(((k-1)*(N*(1-gm)/(K-2)) + gm*N))])
            pt5 = Y[0]
            S = np.sqrt(1/(1-gm)*gm*N/(K-1)*np.sum(np.power(Y[1:]-Y[0],2)))
            hl5 = S/np.sqrt(N)* c_value
            if np.abs(trueth-pt5) < hl5:
                ct5 += 1
            hl5_list.append(hl5)
            result_Kg = []
        print([ct1, ct2, ct3, ct4, ct5, np.average(hl1_list), np.average(hl2_list), np.average(hl3_list), np.average(hl4_list), np.average(hl5_list)])
        result_Kg.append([ct1, ct2, ct3, ct4, ct5, np.average(hl1_list), np.average(hl2_list), np.average(hl3_list), np.average(hl4_list), np.average(hl5_list)])
        print('coverage uncertainty')
        print([1.96*np.sqrt(x/n_sim*(1-x/n_sim))/np.sqrt(n_sim) for x in [ct1,ct2,ct3,ct4,ct5]])
        print('half-length uncertainty')
        print([1.96*np.std(x)/np.sqrt(n_sim) for x in [hl1_list,hl2_list,hl3_list,hl4_list,hl5_list]])
        result_Kg.append([1.96*np.sqrt(x/n_sim*(1-x/n_sim))/np.sqrt(n_sim) for x in [ct1,ct2,ct3,ct4,ct5]])
        result_Kg.append([1.96*np.std(x)/np.sqrt(n_sim) for x in [hl1_list,hl2_list,hl3_list,hl4_list,hl5_list]])
        result_K.append(result_Kg)
    result.append(result_K)


100%|██████████| 100000/100000 [13:41<00:00, 121.77it/s]


[89944, 89933, 90463, 89415, 90665, 0.07784539945388505, 0.07782454787177893, 0.08167820781222913, 0.07849141553761438, 0.08600179364350968]
coverage uncertainty
[0.0018640381757738759, 0.001864943358672751, 0.0018205283892457152, 0.0019068068041728818, 0.0018031550206346657]
half-length uncertainty
[0.00015757292155523233, 0.00015882852826841197, 0.00017957654710070909, 0.00017182855599949577, 0.00020812969891719735]


100%|██████████| 100000/100000 [27:13<00:00, 61.20it/s]


[90010, 89887, 91087, 89615, 90315, 0.07127587049843814, 0.07140976551398671, 0.07622396308509556, 0.07199273737520286, 0.08001464005893394]
coverage uncertainty
[0.0018585925685421212, 0.0018687212233529113, 0.0017660224654657145, 0.0018908177751544432, 0.001833098605432888]
half-length uncertainty
[9.752373419950464e-05, 9.989836075235207e-05, 0.0001217229813680813, 0.00011053698811109768, 0.00017397369130318406]


100%|██████████| 100000/100000 [38:53<00:00, 42.85it/s]


[90064, 90021, 91281, 89892, 90307, 0.06979164039007953, 0.06994194850573565, 0.07459758159599508, 0.07057082125774479, 0.0791986352691488]
coverage uncertainty
[0.0018541184624522777, 0.0018576825395783857, 0.0017485562376493354, 0.0018683111666897452, 0.0018337743146422348]
half-length uncertainty
[7.982337849507255e-05, 8.249948371969204e-05, 0.0001036468061150583, 9.382455087004606e-05, 0.00016997567352513342]
